In [87]:
import math
import requests
import datetime
from statistics import mean
import time
import sys

YEAR = 2019
WEEK = 16
POST_WEEK = 2
LI_RATIO = .1
L_SHARE = LI_RATIO
I_SHARE = 1 - LI_RATIO
TRANSFER_RATIO = 1
MARGIN_LOG = 7

In [88]:
#prev = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=regular')
#prev_post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=postseason')
reg = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=regular')
post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=postseason')
records = {}
games = reg.json() + post.json() #+ prev.json() + prev_post.json()

In [89]:
for game in games:
        
    away_team = game['away_team']
    home_team = game['home_team']
    
    # With this dataset, no conference listed means they are an FCS team. 
    # I treat all FCS teams as the same, likely crappy team. 
    if not game['away_conference']:
        away_team = 'FCS Team'
    if not game['home_conference']:
        home_team = 'FCS Team'   
    
    if away_team not in records:
        records[away_team] = []
    if home_team not in records:
        records[home_team] = []
    
    if ((game['season_type'] != 'postseason'and game['week'] > WEEK) 
        or (game['season_type'] == 'postseason' and game['week'] > POST_WEEK)):
        continue
    
    # For invalid/unplayed games
    if game['away_points'] == None:
        continue
        
    margin = game['away_points'] - game['home_points']
    
    # Going to overtime counts as a one point win
    if len(game['away_line_scores']) > 4:
        margin = margin / abs(margin)
    records[away_team].append((home_team, margin))
    records[home_team].append((away_team, -margin))

In [90]:
records['Clemson']

[('Georgia Tech', 38),
 ('Texas A&M', 14),
 ('Syracuse', 35),
 ('Charlotte', 42),
 ('North Carolina', 1),
 ('Florida State', 31),
 ('Louisville', 35),
 ('Boston College', 52),
 ('FCS Team', 45),
 ('NC State', 45),
 ('Wake Forest', 49),
 ('South Carolina', 35),
 ('Virginia', 45),
 ('Ohio State', 6),
 ('LSU', -17)]

In [102]:
start = time.time()
batch_start = time.time()

rankings = {}
scores = {}
for team in records:
    rankings[team] = 0
    scores[team] = [0]

team_count = len(rankings.keys())

rank_history = [] # used for checking if we have hit a repeating pattern based on ranks
past_averaged_history = '' # used to see if the patterns are broadly repeating
rank_history_set = set() 
score_history = [] # used for easier lookups of what team scores were in said repeating pattern
averaged_score_history = [] 
scores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)
repeat_point = 0
pattern_found = False
# Repeat for 100000 interations, but it should hit a repeating steady state before then
for i in range(1000001):
    for team in records.keys():
        scores[team] = [0] # the first value is the total score
        games = records[team]
        for game in games:
            opponent = game[0]
            margin = game[1]
            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1
            multiplier *= 1000 # just for nicer numbers
            game_score = 0
            if margin < 0:
                game_score -= L_SHARE * rankings[opponent] / team_count
                game_score -= I_SHARE / (team_count - rankings[opponent])
                if rankings[opponent] > rankings[team]:
                    game_score -= TRANSFER_RATIO * L_SHARE * (team_count - rankings[team]) / team_count
                    game_score -= TRANSFER_RATIO * I_SHARE / (1 + rankings[team])
            if margin > 0:
                game_score += L_SHARE * (team_count - rankings[opponent]) / team_count
                game_score += I_SHARE / (1 + rankings[opponent])
                if rankings[opponent] < rankings[team]:
                    game_score += TRANSFER_RATIO * L_SHARE * rankings[team] / team_count
                    game_score += TRANSFER_RATIO * I_SHARE / (team_count - rankings[team])
            scores[team] += [multiplier * game_score]
        if i == 0:
            scores[team][0] = sum(scores[team])
        elif i == 1:
            scores[team][0] = (sum(scores[team]) + score_history[-1][team][0]) / 2
        else:
            scores[team][0] = (sum(scores[team]) 
                               + (2 * score_history[-1][team][0] 
                                  - score_history[-2][team][0])) / 2
    # This is sorted, but adjusting so tied teams have the same (higher) rank
    new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
    #print(new_rank_orders)
    new_ranks = rankings
    new_ranks[new_rank_orders[0]] = 0
    past_rank = 0
    past_score = scores[new_rank_orders[0]][0]
    for rank, team in enumerate(new_rank_orders):
        if (abs(scores[team][0] - past_score) < 0.01):
            new_ranks[team] = past_rank
        else:
            new_ranks[team] = rank
        past_rank = new_ranks[team]
        past_score = scores[team][0]
    # stop if we are repeating ourselves, record this if we are not
    if str(new_ranks) in rank_history_set:
        repeat_point = rank_history.index(str(new_ranks))
        pattern_found = True
        break
    else:
        rank_history += [str(new_ranks)]
        rank_history_set.add(str(new_ranks))
        score_history += [scores.copy()]
        rankings = new_ranks
        
    # stop if we are *broadly* repeating ourselves
    if i%1000 == 0 and i != 0:
        print(i, 'Iterations Took: ' + str(round(time.time() - batch_start)))
        check_start = time.time()
        
        for team in records:
            score = [x[team] for x in score_history]
            scores[team] = list(map(mean, zip(*score)))
            scores[team][0] = sum(scores[team][1:]) 
        new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
        if str(new_rank_orders) == past_averaged_history:
            pattern_found = True
            break
        else:
            averaged_score_history += [scores.copy()]
            past_averaged_history = str(new_rank_orders)
        print(i, 'Check Took: ' + str(round(time.time() - check_start)))
        batch_start = time.time()
        
        
        
    if i>=1000:
        old_news = rank_history[0]
        rank_history_set.remove(old_news)
        del rank_history[0]
        del score_history[0]
            

print(i)

# average across repeated cycled
for team in records:
    score = []
    if pattern_found: 
        for i in range(repeat_point, len(rank_history)):
            score += [score_history[i][team]]
    else:
        score = [x[team] for x in averaged_score_history]
    scores[team] = list(map(mean, zip(*score)))
    scores[team][0] = sum(scores[team][1:]) 
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ('My Computer Poll!\n'
          + 'It only takes into account games played this season, looking at margin and quality of wins and losses individually:\n\n'
          + '|Rank|Score|Team|Breakdown|\n'
          + '|-|-|-|-|\n')
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(0, len(scores[team]) - 1):
        explanation += (str(round(scores[team][j + 1])) + ' from #'
                        + str(new_rank_orders.index(records[team][j][0]) + 1) + ' '
                        + str(records[team][j][0]) 
                        + '(' + str(records[team][j][1]) + '), ')
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + explanation[0:-2] + '\n')
    output += line
print('Completed in ' + str(time.time() - start) + ' seconds')
print(output)
with open('Output - '
        + str(YEAR) 
          + '-' 
          + str(WEEK) + '.md', 'w') as text_file:
    text_file.write(output)

1000 Iterations Took: 13
1000 Check Took: 6
2000 Iterations Took: 13
2000 Check Took: 4
3000 Iterations Took: 5
3000 Check Took: 3
4000 Iterations Took: 4
4000 Check Took: 3
5000 Iterations Took: 4
5000 Check Took: 3
6000 Iterations Took: 4
6000 Check Took: 3
7000 Iterations Took: 4
7000 Check Took: 3
8000 Iterations Took: 4
8000 Check Took: 3
9000 Iterations Took: 5
9000 Check Took: 3
10000 Iterations Took: 4
10000 Check Took: 3
11000 Iterations Took: 4
11000 Check Took: 3
12000 Iterations Took: 4
12000 Check Took: 3
13000 Iterations Took: 4
13000 Check Took: 3
14000 Iterations Took: 4
14000 Check Took: 3
15000 Iterations Took: 4
15000 Check Took: 3
16000 Iterations Took: 4
16000 Check Took: 3
17000 Iterations Took: 4
17000 Check Took: 3
18000 Iterations Took: 4
18000 Check Took: 3
19000 Iterations Took: 4
19000 Check Took: 3
20000 Iterations Took: 4
20000 Check Took: 3
21000 Iterations Took: 4
21000 Check Took: 3
22000 Iterations Took: 4
22000 Check Took: 3
23000 Iterations Took: 4
2

180000 Iterations Took: 5
180000 Check Took: 3
181000 Iterations Took: 5
181000 Check Took: 3
182000 Iterations Took: 4
182000 Check Took: 3
183000 Iterations Took: 4
183000 Check Took: 3
184000 Iterations Took: 4
184000 Check Took: 3
185000 Iterations Took: 5
185000 Check Took: 3
186000 Iterations Took: 4
186000 Check Took: 3
187000 Iterations Took: 5
187000 Check Took: 3
188000 Iterations Took: 5
188000 Check Took: 3
189000 Iterations Took: 5
189000 Check Took: 3
190000 Iterations Took: 5
190000 Check Took: 3
191000 Iterations Took: 4
191000 Check Took: 3
192000 Iterations Took: 5
192000 Check Took: 3
193000 Iterations Took: 5
193000 Check Took: 3
194000 Iterations Took: 5
194000 Check Took: 3
195000 Iterations Took: 5
195000 Check Took: 3
196000 Iterations Took: 5
196000 Check Took: 3
197000 Iterations Took: 5
197000 Check Took: 3
198000 Iterations Took: 4
198000 Check Took: 3
199000 Iterations Took: 4
199000 Check Took: 3
200000 Iterations Took: 4
200000 Check Took: 3
201000 Iterat

354000 Check Took: 3
355000 Iterations Took: 4
355000 Check Took: 3
356000 Iterations Took: 5
356000 Check Took: 3
357000 Iterations Took: 5
357000 Check Took: 3
358000 Iterations Took: 5
358000 Check Took: 3
359000 Iterations Took: 5
359000 Check Took: 3
360000 Iterations Took: 5
360000 Check Took: 3
361000 Iterations Took: 5
361000 Check Took: 3
362000 Iterations Took: 4
362000 Check Took: 3
363000 Iterations Took: 4
363000 Check Took: 3
364000 Iterations Took: 5
364000 Check Took: 3
365000 Iterations Took: 4
365000 Check Took: 3
366000 Iterations Took: 4
366000 Check Took: 3
367000 Iterations Took: 4
367000 Check Took: 3
368000 Iterations Took: 5
368000 Check Took: 3
369000 Iterations Took: 5
369000 Check Took: 3
370000 Iterations Took: 5
370000 Check Took: 3
371000 Iterations Took: 4
371000 Check Took: 3
372000 Iterations Took: 4
372000 Check Took: 3
373000 Iterations Took: 4
373000 Check Took: 3
374000 Iterations Took: 5
374000 Check Took: 3
375000 Iterations Took: 5
375000 Check 

529000 Iterations Took: 5
529000 Check Took: 3
530000 Iterations Took: 5
530000 Check Took: 3
531000 Iterations Took: 4
531000 Check Took: 3
532000 Iterations Took: 4
532000 Check Took: 3
533000 Iterations Took: 5
533000 Check Took: 3
534000 Iterations Took: 5
534000 Check Took: 3
535000 Iterations Took: 4
535000 Check Took: 3
536000 Iterations Took: 4
536000 Check Took: 3
537000 Iterations Took: 4
537000 Check Took: 3
538000 Iterations Took: 4
538000 Check Took: 3
539000 Iterations Took: 4
539000 Check Took: 3
540000 Iterations Took: 4
540000 Check Took: 3
541000 Iterations Took: 4
541000 Check Took: 3
542000 Iterations Took: 4
542000 Check Took: 3
543000 Iterations Took: 4
543000 Check Took: 3
544000 Iterations Took: 4
544000 Check Took: 3
545000 Iterations Took: 4
545000 Check Took: 3
546000 Iterations Took: 5
546000 Check Took: 3
547000 Iterations Took: 4
547000 Check Took: 3
548000 Iterations Took: 4
548000 Check Took: 3
549000 Iterations Took: 4
549000 Check Took: 3
550000 Iterat

703000 Check Took: 3
704000 Iterations Took: 4
704000 Check Took: 3
705000 Iterations Took: 4
705000 Check Took: 3
706000 Iterations Took: 4
706000 Check Took: 3
707000 Iterations Took: 4
707000 Check Took: 3
708000 Iterations Took: 4
708000 Check Took: 3
709000 Iterations Took: 5
709000 Check Took: 3
710000 Iterations Took: 4
710000 Check Took: 3
711000 Iterations Took: 5
711000 Check Took: 3
712000 Iterations Took: 4
712000 Check Took: 3
713000 Iterations Took: 4
713000 Check Took: 3
714000 Iterations Took: 4
714000 Check Took: 3
715000 Iterations Took: 4
715000 Check Took: 3
716000 Iterations Took: 4
716000 Check Took: 3
717000 Iterations Took: 4
717000 Check Took: 3
718000 Iterations Took: 5
718000 Check Took: 3
719000 Iterations Took: 4
719000 Check Took: 3
720000 Iterations Took: 4
720000 Check Took: 3
721000 Iterations Took: 4
721000 Check Took: 3
722000 Iterations Took: 4
722000 Check Took: 3
723000 Iterations Took: 5
723000 Check Took: 3
724000 Iterations Took: 5
724000 Check 

878000 Iterations Took: 4
878000 Check Took: 3
879000 Iterations Took: 4
879000 Check Took: 3
880000 Iterations Took: 4
880000 Check Took: 3
881000 Iterations Took: 5
881000 Check Took: 3
882000 Iterations Took: 4
882000 Check Took: 3
883000 Iterations Took: 5
883000 Check Took: 3
884000 Iterations Took: 4
884000 Check Took: 3
885000 Iterations Took: 4
885000 Check Took: 3
886000 Iterations Took: 5
886000 Check Took: 3
887000 Iterations Took: 5
887000 Check Took: 3
888000 Iterations Took: 5
888000 Check Took: 3
889000 Iterations Took: 5
889000 Check Took: 3
890000 Iterations Took: 5
890000 Check Took: 3
891000 Iterations Took: 5
891000 Check Took: 3
892000 Iterations Took: 4
892000 Check Took: 3
893000 Iterations Took: 4
893000 Check Took: 3
894000 Iterations Took: 4
894000 Check Took: 3
895000 Iterations Took: 4
895000 Check Took: 4
896000 Iterations Took: 5
896000 Check Took: 3
897000 Iterations Took: 5
897000 Check Took: 3
898000 Iterations Took: 5
898000 Check Took: 3
899000 Iterat

In [103]:
ranks = []
for i in range (5):
    ranks.append(sorted(scores, key=averaged_score_history[-i].__getitem__, reverse=True))
for i in range(len(ranks[0])):
    row = str(i) + ' '
    for j in range (5):
        row += ranks[j][i] + ' '
    print(row)

0 LSU LSU LSU LSU LSU 
1 Ohio State Ohio State Ohio State Ohio State Ohio State 
2 Clemson Clemson Clemson Clemson Clemson 
3 Georgia Georgia Georgia Georgia Georgia 
4 Florida Florida Florida Florida Florida 
5 Oregon Oregon Oregon Oregon Oregon 
6 Penn State Penn State Penn State Penn State Penn State 
7 Alabama Wisconsin Wisconsin Wisconsin Wisconsin 
8 Wisconsin Alabama Alabama Alabama Alabama 
9 Oklahoma Oklahoma Oklahoma Oklahoma Oklahoma 
10 Notre Dame Notre Dame Notre Dame Notre Dame Notre Dame 
11 Memphis Memphis Memphis Memphis Memphis 
12 Auburn Auburn Auburn Auburn Auburn 
13 Baylor Baylor Baylor Baylor Baylor 
14 Minnesota Minnesota Minnesota Minnesota Minnesota 
15 Utah Utah Utah Utah Utah 
16 Cincinnati Cincinnati Cincinnati Cincinnati Cincinnati 
17 Navy Navy Navy Navy Navy 
18 Michigan Michigan Michigan Michigan Michigan 
19 Iowa Iowa Iowa Iowa Iowa 
20 Appalachian State Appalachian State Appalachian State Appalachian State Appalachian State 
21 Boise State USC USC USC